In [57]:
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2021-CS109A/master/"
    "themes/static/css/cs109.css"
).text
HTML(styles)

In [58]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import (LogisticRegression, LinearRegression)
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier

# pandas tricks for better display
pd.options.display.max_columns = 50  
pd.options.display.max_rows = 500     
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

# pandas tricks for better display
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

# Silence FutureWarnings
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

# Problem Statement (Caleb)

1. Determine and train the best model for classifying galaxies on morphological type (TT response variable) and on Hubble type (spiral, lenticular, irregular), and determine which physical parameters are most important for classification.

2. Recovering known astronomical relationships (e.g. Hubble Constant) from the data and giving a physical sense for our findings.


# Get Data

In [59]:
# Read in the data
processed_galaxies_df = pd.read_csv('../data/preprocessed_data.csv')
processed_galaxies_df= processed_galaxies_df.drop(['Unnamed: 0', 'index'], axis=1)

In [60]:
# Read in data that includes indicator for imputed values
missingness = pd.read_csv('../data/missingness.csv')
missingness= missingness.drop(['Unnamed: 0'], axis=1)

# EDA (John, Adam, Caleb)

# Model Discussion (John, Adam)

Which models could work and why?

* __Multiple Logistic Regression with LASSO Regularization__: Useful in multiclass problems on non-ordinal, categorical response variables. Good for interpretability since we get coefficient estimates to relate predictors and the log-odds of belonging to a specific class; when used with LASSO regularization, parameters are shrunk to zero, preserving the interpretability.

* __Random Forest__: Useful in multi-class, classification problems. Random forests help reduce collinearity between predictors with the randomness involved in decision tree splits. They may also find more complex decision boundaries than just logistic regression due to the non-parametric approach to modeling. Lastly, there is a very straightforward method for feature importance.

* __Gradient Boosting__: Useful in multi-class, classification problems. GB provides good performance with an inbalanced data sets because it trains itself on the residuals; i.e. underrepresented data will have higher error, and thus the model will focus more on those data points on subsequent interations. Lastly, GB may also find more complex decision boundaries than just logistic regression due to the non-parametric approach to modeling.


# Hubble Class

## Model Selection and Tuning

#### Data Preprocessing

1. Get Hubble class (spiral, irregular, lenticular)
2. Split into train and test sets
3. Standardization of quantitative predictors

In [61]:
# Define a variable 'gal_type' that stores the type of galaxy 
processed_galaxies_df['hubble_class'] = 0
processed_galaxies_df['hubble_class'][(processed_galaxies_df['TT'] >= 0) & (processed_galaxies_df['TT'] <= 7)] = 1
processed_galaxies_df['hubble_class'][processed_galaxies_df['TT'] >  7] = 2
## ===== CODE: (0, 1, 2) = (lenticular, spiral, irregular) ===== ##

# Print the percent class breakdowns for each Hubble class
print("Portion Lenticular:", round(np.mean(processed_galaxies_df['hubble_class'] == 0), 3))
print("Portion Spiral:", round(np.mean(processed_galaxies_df['hubble_class'] == 1), 3))
print("Portion Irregular:", round(np.mean(processed_galaxies_df['hubble_class'] == 2), 3))

Portion Lenticular: 0.197
Portion Spiral: 0.124
Portion Irregular: 0.679


/var/folders/b7/7r72r1kn6x596_mb7pz6xxfc0000gn/T/ipykernel_86581/1050591817.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_galaxies_df['hubble_class'][(processed_galaxies_df['TT'] >= 0) & (processed_galaxies_df['TT'] <= 7)] = 1
/var/folders/b7/7r72r1kn6x596_mb7pz6xxfc0000gn/T/ipykernel_86581/1050591817.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_galaxies_df['hubble_class'][processed_galaxies_df['TT'] >  7] = 2


In [62]:
# Instantiate the list of the quantitative predictors
quant_preds = processed_galaxies_df.select_dtypes(include=['number']).columns.drop(['TT', 'hubble_class'])

# Split into train and test sets for hubble class
X_train, X_test, y_train_hubble, y_test_hubble = train_test_split(processed_galaxies_df[quant_preds], processed_galaxies_df['hubble_class'], random_state=109, test_size=0.2, shuffle=True)

# Instantiate a scaler
standardscaler = StandardScaler().fit(X_train)

# Standardize the data
X_train_std = standardscaler.transform(X_train)
X_test_std = standardscaler.transform(X_test)

# Define a dictionary to store the train and test accuracies of the optimal models throughout this notebook
accuracies = {}

#### Multiple Logistic Regression with LASSO Regularization, Tuned with CV (k=25), Not-SMOTE

In [63]:
# Parameters to tune with SMOTE and CV
param_grid = {'C': np.logspace(-4, 4, 20)}

# Base classifier
logreg_base = LogisticRegression(penalty='l1', solver='liblinear', multi_class='ovr', random_state=109)

# Perform the cross_validation
grid_search = GridSearchCV(logreg_base, param_grid, cv=25, scoring='accuracy').fit(X_train_std, y_train_hubble)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Fit the best model from the cross validation and compute the train and test accuracies
best_logreg = LogisticRegression(penalty='l1', solver='liblinear', C=grid_search.best_params_['C']).fit(X_train_std, y_train_hubble)
best_logreg_train_acc = best_logreg.score(X_train_std, y_train_hubble)
best_logreg_test_acc = best_logreg.score(X_test_std, y_test_hubble)
print("Logreg Train Accuracy:", best_logreg_train_acc)
print("Logreg Test Accuracy:", best_logreg_test_acc)

# Store accuracies in the accuracies dictionary
accuracies['Logreg'] = {
    'train': best_logreg_train_acc,
    'test': best_logreg_test_acc,
    'val': grid_search.best_score_
}


Best Parameters: {'C': 11.288378916846883}
Best Cross-Validation Score: 0.8723809523809523
Logreg Train Accuracy: 0.8914616497829233
Logreg Test Accuracy: 0.8901734104046243


#### Multiple Logistic Regression with LASSO Regularization, Tuned with CV (k=25), With SMOTE

In [64]:
# Parameters to tune with CV
param_grid = {'log_reg__C': np.logspace(-4, 4, 20)}

# Pipeline for CV with SMOTE
pipeline = Pipeline([
    ('smote', SMOTE(random_state=109)),
    ('log_reg', LogisticRegression(penalty='l1', solver='liblinear', multi_class='ovr', random_state=109))
])    

# Perform the cross_validation
grid_search = GridSearchCV(pipeline, param_grid, cv=25, scoring='accuracy').fit(X_train_std, y_train_hubble)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# Fit the best model from the cross validation and compute the train and test accuracies
best_logreg = LogisticRegression(penalty='l1', solver='liblinear', C=grid_search.best_params_['log_reg__C']).fit(X_train_std, y_train_hubble)
best_logreg_train_acc_SMOTE = best_logreg.score(X_train_std, y_train_hubble)
best_logreg_test_acc_SMOTE = best_logreg.score(X_test_std, y_test_hubble)
print("Logistic-SMOTE Train Accuracy:", best_logreg_train_acc_SMOTE)
print("Logistic-SMOTE Test Accuracy:", best_logreg_test_acc_SMOTE)


# Store accuracies in the accuracies dictionary
accuracies['Logreg-SMOTE'] = {
    'train': best_logreg_train_acc_SMOTE,
    'test': best_logreg_test_acc_SMOTE,
    'val': grid_search.best_score_
}


Best Parameters: {'log_reg__C': 4.281332398719396}
Best Cross-Validation Score: 0.8577777777777778
Logistic-SMOTE Train Accuracy: 0.8914616497829233
Logistic-SMOTE Test Accuracy: 0.8959537572254336


#### Random Forest, Tuned with CV (k=25), Not-SMOTE

In [65]:
# Parameters to tune with SMOTE and CV
param_grid = {'max_depth': np.arange(5, 15, 1), 'criterion': ['gini', 'entropy'], 'min_samples_split': [2, 5, 10]}

# Base classifier
rf_base = RandomForestClassifier(n_estimators=100, random_state=109)

# Perform the cross_validation
grid_search = GridSearchCV(rf_base, param_grid, cv=25, scoring='accuracy').fit(X_train_std, y_train_hubble)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)
best_params = grid_search.best_params_

# Fit the best model based on the CV, and calculate the train and test accuracies of best estimator
best_rf = RandomForestClassifier(n_estimators=100, max_depth=best_params['max_depth'], criterion=best_params['criterion'], min_samples_split=best_params['min_samples_split'], random_state=109).fit(X_train_std, y_train_hubble)
best_rf_train_acc = best_rf.score(X_train_std, y_train_hubble)
best_rf_test_acc = best_rf.score(X_test_std, y_test_hubble)
print("RF Train Accuracy:", best_rf_train_acc)
print("RF Test Accuracy:", best_rf_test_acc)

accuracies['random_forest'] = {
    'train': best_rf_train_acc,
    'test' : best_rf_test_acc,
    'val': grid_search.best_score_
}


Best Parameters: {'criterion': 'entropy', 'max_depth': 11, 'min_samples_split': 2}
Best Cross-Validation Score: 0.8984656084656085
RF Train Accuracy: 1.0
RF Test Accuracy: 0.8786127167630058


#### Random Forest, Tuned with CV (k=25), With SMOTE

In [66]:
# Parameters to tune with SMOTE and CV
param_grid = {'classifier__max_depth': np.arange(5, 15, 1), 'classifier__criterion': ['gini', 'entropy'], 'classifier__min_samples_split': [2, 5, 10]}

# Pipeline for CV with SMOTE
pipeline = Pipeline([
    ('smote', SMOTE(random_state=109)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=109))
])    

# Perform the cross_validation
grid_search = GridSearchCV(pipeline, param_grid, cv=25, scoring='accuracy').fit(X_train_std, y_train_hubble)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)
best_params = grid_search.best_params_

# Fit the best model based on the CV, and calculate the train and test accuracies of best estimator
best_rf = RandomForestClassifier(n_estimators=100, max_depth=best_params['classifier__max_depth'], criterion=best_params['classifier__criterion'], min_samples_split=best_params['classifier__min_samples_split'], random_state=109).fit(X_train_std, y_train_hubble)
best_rf_train_acc_SMOTE = best_rf.score(X_train_std, y_train_hubble)
best_rf_test_acc_SMOTE = best_rf.score(X_test_std, y_test_hubble)
print("RF-SMOTE Train Accuracy:", best_rf_train_acc_SMOTE)
print("RF-SMOTE Test Accuracy:", best_rf_test_acc_SMOTE)

accuracies['random_forest-SMOTE'] = {
    'train': best_rf_train_acc_SMOTE,
    'test' : best_rf_test_acc_SMOTE,
    'val' : grid_search.best_score_
}

Best Parameters: {'classifier__criterion': 'entropy', 'classifier__max_depth': 9, 'classifier__min_samples_split': 2}
Best Cross-Validation Score: 0.8912698412698412
RF-SMOTE Train Accuracy: 1.0
RF-SMOTE Test Accuracy: 0.8786127167630058


#### sklearn Gradient Boosting, Tuned with CV (k=10), Not-SMOTE

In [70]:
# Parameters to tune with CV
param_grid = {"max_depth": [2, 3, 4], "learning_rate": [0.001, 0.01, 0.1], "n_estimators": [50, 100, 150]}

# base boosting classifier
boost_base = GradientBoostingClassifier(random_state=109)

# Perform grid search
grid_search = GridSearchCV(boost_base, param_grid, cv=10, scoring='accuracy', verbose=1).fit(X_train_std, y_train_hubble)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)
best_params = grid_search.best_params_

# Fit the best model based on the CV, and calculate the train and test accuracies of best estimator
best_booster = GradientBoostingClassifier(max_depth=best_params['max_depth'], learning_rate=best_params['learning_rate'], n_estimators=best_params['n_estimators'], random_state=109).fit(X_train_std, y_train_hubble)
best_booster_train_acc = best_booster.score(X_train_std, y_train_hubble)
best_booster_test_acc = best_booster.score(X_test_std, y_test_hubble)
print("GradBoost Train Accuracy:", best_booster_train_acc)
print("GradBoost Test Accuracy:", best_booster_test_acc)

accuracies['skl_grad_boost'] = {
    'train': best_booster_train_acc,
    'test' : best_booster_test_acc,
    'val' : grid_search.best_score_
}


Fitting 10 folds for each of 27 candidates, totalling 270 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
Best Cross-Validation Score: 0.9102691511387162
GradBoost Train Accuracy: 1.0
GradBoost Test Accuracy: 0.9075144508670521


#### sklearn Gradient Boosting, Tuned with CV (k=10), With-SMOTE

In [68]:
param_grid = { "classifier__max_depth": [2, 3, 4], "classifier__learning_rate": [0.001, 0.01, 0.1], "classifier__n_estimators": [50, 100, 150]}

# Pipeline for CV with SMOTE
pipeline = Pipeline([
    ('smote', SMOTE(random_state=109)),
    ('classifier', GradientBoostingClassifier(random_state=109))
])

# Grid serach CV
grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='accuracy', verbose=1).fit(X_train_std, y_train_hubble)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)
best_params = grid_search.best_params_

# Fit the best model based on the CV, and calculate the train and test accuracies of best estimator
best_booster = GradientBoostingClassifier(max_depth=best_params['classifier__max_depth'], learning_rate=best_params['classifier__learning_rate'], n_estimators=best_params['classifier__n_estimators'], random_state=109).fit(X_train_std, y_train_hubble)
best_booster_train_acc_SMOTE = best_booster.score(X_train_std, y_train_hubble)
best_booster_test_acc_SMOTE = best_booster.score(X_test_std, y_test_hubble)
print("GradBoost Train Accuracy:", best_booster_train_acc_SMOTE)
print("GradBoost Test Accuracy:", best_booster_test_acc_SMOTE)

accuracies['skl_grad_boost-SMOTE'] = {
    'train': best_booster_train_acc_SMOTE,
    'test' : best_booster_test_acc_SMOTE,
    'val': grid_search.best_score_
}


Fitting 10 folds for each of 27 candidates, totalling 270 fits
Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 150}
Best Cross-Validation Score: 0.916024844720497
GradBoost Train Accuracy: 1.0
GradBoost Test Accuracy: 0.9017341040462428


#### Display the Accuracies of Each Model

In [71]:
display(pd.DataFrame(accuracies).T)

,train,test,val
Logreg,0.891,0.890,0.872
Logreg-SMOTE,0.891,0.896,0.858
random_forest,1.000,0.879,0.898
random_forest-SMOTE,1.000,0.879,0.891
skl_grad_boost,1.000,0.908,0.910
skl_grad_boost-SMOTE,1.000,0.902,0.916
